<a href="https://colab.research.google.com/github/Aakaey181/RA_Porject_ML1/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install streamlit streamlit-drawable-canvas
!pip install streamlit pyngrok

In [47]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
%%writefile app.py
import streamlit as st
from streamlit_drawable_canvas import st_canvas
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf


# Load the trained model (adjust the path accordingly)
model = tf.keras.models.load_model("drive/MyDrive/ra/trained_models/clean_mobilenet_binary_classifier.h5")

st.title("Zebra Crossing Detector")

# Upload image
uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image",  use_container_width=True)

    # Create an interactive canvas for drawing
    canvas_result = st_canvas(
        fill_color="rgba(255, 165, 0, 0.3)",  # Semi-transparent fill
        stroke_width=3,
        stroke_color="#e00",
        update_streamlit=True,
        height=image.height,
        width=image.width,
        drawing_mode="rect",  # Enable rectangle drawing
        key="canvas",
    )

    # Process the drawn rectangle
    if canvas_result.json_data is not None:
        objects = canvas_result.json_data.get("objects", [])
        if objects:
            st.write("Drawn objects:", objects)
            # Assume the first rectangle is the selected region
            rect = objects[0]
            left = int(rect.get("left", 0))
            top = int(rect.get("top", 0))
            width = int(rect.get("width", 0))
            height = int(rect.get("height", 0))
            st.write(f"Bounding box: Left={left}, Top={top}, Width={width}, Height={height}")

            # Crop the image using the bounding box coordinates
            image_np = np.array(image)
            cropped = image_np[top:top+height, left:left+width]
            st.image(cropped, caption="Cropped Region",  use_container_width=True)

            # Preprocess the cropped image
            cropped_resized = cv2.resize(cropped, (224, 224))
            cropped_resized = cropped_resized / 255.0
            cropped_resized = np.expand_dims(cropped_resized, axis=0)

            # Predict with the model
            prob = model.predict(cropped_resized)[0][0]
            confidence = prob * 100
            st.write(f"Prediction: {confidence:.2f}% confidence that a zebra crossing is present.")

Overwriting app.py


In [51]:
!pip install streamlit streamlit-drawable-canvas pyngrok
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [49]:
!ngrok authtoken 2vCjd1FQD3nj46KDGwA02cD8fmE_2JU7HFoujPjiYnTW9xDDR

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [58]:
from pyngrok import ngrok

# Specify the protocol as http for the tunnel
public_url = ngrok.connect(8501, "http")

print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://6319-34-75-82-143.ngrok-free.app" -> "http://localhost:8501"
